In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
#from keras.models import model_from_json
import numpy as np
import os

import matplotlib.pyplot as plt
import matplotlib.patches as patches

#from tensorflow.keras.utils import np_utils # To transform labels in categorical
#import tensorflow.keras.np_utils
from keras.utils import to_categorical
from tensorflow.keras.datasets import mnist # To load the dataset
from tensorflow.keras import backend as K
import tensorflow.keras as keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten,Activation,AveragePooling2D, Normalization
#from tensorflow.keras.layers.normalization import batch_normalization as BatchNormalization

In [2]:
import tensorflow as tf

In [3]:
(trainX, trainY), (testX, testY) = mnist.load_data()
# reshape dataset to have a single channel
trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
testX = testX.reshape((testX.shape[0], 28, 28, 1))

In [4]:
trainY = to_categorical(trainY,10)
testY = to_categorical(testY,10)

In [5]:
# convert from integers to floats
train_norm = trainX.astype('float32')
test_norm = testX.astype('float32')
# normalize to range 0-1
train_norm = train_norm / 255.0
test_norm = test_norm / 255.0

## CNN

In [6]:
model = Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu', kernel_initializer='he_uniform'))
model.add(keras.layers.Dense(10, activation='softmax'))
# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
history = model.fit(train_norm, trainY, epochs=5, batch_size=32, validation_data=(test_norm, testY), verbose=1)

Epoch 1/5
1119/1875 [================>.............] - ETA: 15s - loss: 0.1920 - accuracy: 0.9403

KeyboardInterrupt: 

In [ ]:
model.save('cnn_model.h5')

# FFNN

In [ ]:
model = Sequential()
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu', kernel_initializer='he_uniform'))
model.add(keras.layers.Dense(10, activation='softmax'))
# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_norm, trainY, epochs=5, batch_size=32, validation_data=(test_norm, testY), verbose=1)

In [ ]:
model.save('ffnn_model.h5')

## Model with data augmentation

In [ ]:
model = Sequential()

model.add(Conv2D(32,(3,3), input_shape=(28,28,1), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(32,(3,3), input_shape=(28,28,1), padding='same'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(64,(3,3), padding='same'))
model.add(Activation('relu'))

model.add(AveragePooling2D(pool_size=(2)))

model.add(Conv2D(128,(3,3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(128,(3,3), padding='same'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256,(3,3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(256,(3,3), padding='same'))
model.add(Activation('relu'))

model.add(AveragePooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.summary()

In [ ]:
datagen = keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.4,
    height_shift_range=0.4,
    validation_split=0.2)

In [ ]:
datagen.fit(train_norm)

train_generator = datagen.flow(train_norm, trainY, batch_size=64, shuffle=True, 
                               seed=2, save_to_dir=None, subset='training')

validation_generator = datagen.flow(train_norm, trainY, batch_size=64, shuffle=True, 
                               seed=2, save_to_dir=None, subset='validation')

This training may take longer than the others. You will need roughly 5-10 epochs to get a good enough result.

In [ ]:
epochs = 5
model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
history = model.fit(train_generator, steps_per_epoch = 600, epochs=epochs,
          validation_data = validation_generator,
          validation_steps = 150)

In [ ]:
model.save('cnn_augm_model.h5')

In [ ]:
# Loss Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['loss'],'black',linewidth=3.0)
plt.plot(history.history['val_loss'],'black',ls = '--', linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
#plt.title('Loss Curves',fontsize=16)


In [ ]:
# Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['accuracy'],'black',linewidth=3.0)
plt.plot(history.history['val_accuracy'],'black',ls = '--',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18, 
           loc = 'lower right')
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
#plt.title('Accuracy Curves',fontsize=16)
